## Submit

## data

In [1]:
from utils.submit import get_test_dataloader

test_loader, test_dataset = get_test_dataloader()

In [2]:
print('=============test_loader =============')
for data in test_loader:
    print(data)
    print(data.shape)
    break

=============test_loader =============
tensor([[[[-1.0904, -0.8507, -0.8164,  ..., -1.8782, -1.8782, -1.8439],
          [-0.6281, -0.4054, -0.5424,  ..., -1.8782, -1.8782, -1.8097],
          [ 0.7077,  0.2453, -0.7822,  ..., -1.6555, -1.6384, -1.5870],
          ...,
          [-1.7925, -1.7069, -1.6042,  ..., -0.7479, -0.7650, -0.8678],
          [-1.7583, -1.7583, -1.6898,  ..., -0.7993, -1.1247, -1.2617],
          [-1.6042, -1.7583, -1.7754,  ..., -1.2788, -1.2103, -1.1418]],

         [[-0.7052, -0.4601, -0.4776,  ..., -1.6681, -1.6681, -1.6506],
          [-0.2150, -0.0049, -0.1800,  ..., -1.7031, -1.6681, -1.6331],
          [ 1.1331,  0.6779, -0.4076,  ..., -1.5280, -1.4580, -1.4405],
          ...,
          [-1.7381, -1.6506, -1.5805,  ..., -0.8627, -0.8803, -1.0028],
          [-1.7381, -1.7381, -1.6681,  ..., -0.9153, -1.2479, -1.4055],
          [-1.6155, -1.7731, -1.7906,  ..., -1.3704, -1.3179, -1.2479]],

         [[-0.6890, -0.4275, -0.3753,  ..., -1.5604, -1.5604, -

## Predict

In [4]:
import torchvision.models as models
import torch
import torch.nn as nn

# 采用本地训练好的模型
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet50.fc = nn.Linear(2048,102)
for param in resnet50.parameters():
    param.requires_grad = False
for param in resnet50.fc.parameters():
    param.requires_grad = True
resnet50.load_state_dict(torch.load('resnet50_epoch10.pth', map_location='cpu'))

<All keys matched successfully>

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# 将预测结果写入列表中
pred_list = []
resnet50.eval()
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        output = resnet50(data)
        pred_list.extend(output.argmax(dim=-1).tolist())

In [7]:
len(pred_list)

1145

In [8]:
with open("./data/dataset/test.txt", "r", encoding="utf-8") as f:
    test_text = f.readlines()
len(test_text)

1145

In [9]:
with open('./data/result.txt', 'w', encoding='utf-8') as f_result:
    for i, pred in enumerate(pred_list):
        f_result.write(test_text[i].strip() + '\t' + str(pred) + '\n')